In [ ]:
import matplotlib.pyplot as plt
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import ToTensor

In [ ]:
train_data = CIFAR10(
    root = './',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = CIFAR10(
    root = './',
    train = False,
    download = True,
    transform = ToTensor()
)

In [ ]:
for temp in range(9):
    plt.subplot(3,3,temp+1)
    plt.imshow(train_data.data[temp])
plt.show()

# 데이터 증강
- 회전
- 사이즈 변형
- 좌우, 상하 반전
- 이동
- crop (전체 이미지에서 잘라쓰기)

#### 색깔의 영향을 받지 않기 위해 정규화 작업을 해야한다

In [ ]:
from torchvision.transforms import Compose
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
import torchvision.transforms as T

test_transforms = Compose(
    [
        T.ToPILImage(),
        # 랜덤으로 이미지를 자르고 패딩을 4로 처리
        RandomCrop((32,32), padding = 4),
        # 0.5의 확률로 y축을 기준으로 반전 시킨다.
        RandomHorizontalFlip(p = 0.5),
        
        T.ToTensor(),
        # R,G,B의 평균, 표준편차
        Normalize(mean = (0.4733, 0.4643, 0.4465), std = (0.251, 0.253, 0.261)),
        T.ToPILImage()
    ]
)

train_data = CIFAR10(
    root = './',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = CIFAR10(
    root = './',
    train = False,
    download = True,
    transform = ToTensor()
)

for temp in range(9):
    plt.subplot(3,3,temp+1)
    plt.imshow(test_transforms(train_data.data[temp]))
plt.show()

## 이미지 R,G,B 평균 표준편차 확인해보기

In [ ]:
import torch

imgs = []

for temp in train_data:
    imgs.append(temp[0])

# 이미지들을 하나로 합쳐서 통합을 시킨다.
imgs = torch.stack(imgs, dim = 0).numpy()

# 0번째는 R 1은 G 2는 B
mean_r = imgs[:,0:,:].mean()
mean_g = imgs[:,1:,:].mean()
mean_b = imgs[:,2:,:].mean()
print(mean_r, mean_g, mean_b)

In [ ]:
std_r = imgs[:,0:,:].std()
std_g = imgs[:,1:,:].std()
std_b = imgs[:,2:,:].std()
print(std_r, std_g, std_b)

# 시드 값 고정
- 4가지 방법

In [ ]:
import torch
import numpy as np

torch.manual_seed(100)
torch.cuda.manual_seed(100)
torch.cuda.manual_seed_all(100)
np.random.seed(100)



# VGG 모델 구현

In [1]:
import torch
import torch.nn as nn

In [2]:
class BasicBlock(nn.Module):
    def __init__(self,in_channels,out_channels,hidden_dim):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(hidden_dim, out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        return x


class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN,self).__init__()

        self.block1 = BasicBlock(in_channels=3,out_channels=32,hidden_dim=16)
        self.block2 = BasicBlock(in_channels=32,out_channels=128,hidden_dim=64)
        self.block3 = BasicBlock(in_channels=128,out_channels=256,hidden_dim=128)

        self.fc1 = nn.Linear(in_features=4096, out_features=2048)
        self.fc2 = nn.Linear(in_features=2048, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=num_classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, start_dim=1)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)

        return x





In [ ]:
# class BasicBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, hidden_dim):
#         super(BasicBlock, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels, hidden_dim, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(hidden_dim, out_channels, kernel_size = 3, padding = 1)
#         self.relu = nn.ReLU()
#         self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu(x)
#         x = self.conv2(x)
#         x = self.relu(x)
#         x = self.pool(x)
        
#         return x
    
# class CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(CNN, self).__init__()
        
#         # 스트라이드 출력 크기 계산 알아야함. 입 출력 데이터가 어케 변하는지
#         ## (w + 2p - fw)/s +1 계산으로 인해 32이지만 maxpool로 인해 절반 나가리
#         ### 32*32
#         self.block1 = BasicBlock(in_channels=3,out_channels=32,hidden_dim=16)
#         ### 16*16
#         self.block2 = BasicBlock(in_channels=32, out_channels=128, hidden_dim=64)
#         ### 8*8의 
#         self.block3 = BasicBlock(in_channels=128, out_channels=256, hidden_dim=128)
#         ### 최종적으로 4*4 256이 나오게 된다.
        
#         # 4*4*256
#         self.fc1 = nn.Linear(in_features=4096, out_features=2048)
#         self.fc2 = nn.Linear(in_features=2048, out_features=256)
#         self.fc3 = nn.Linear(in_features=256, out_features=num_classes)

#         self.relu = nn.ReLU()

#     def forward(self, x):
#         x = self.block1(x)
#         x = self.block2(x)
#         x = self.block3(x)
#         x = torch.flatten(x, start_dim = 1)
        
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         x = self.relu(x)
#         x = self.fc3(x)
        
#         return x 

In [3]:
from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose
from torchvision.transforms import RandomCrop, RandomHorizontalFlip,Normalize
import torchvision.transforms as T

In [4]:
transforms = Compose([
    RandomCrop((32,32),padding=4),
    RandomHorizontalFlip(p=0.5),
    T.ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465),std=(0.247,0.243,0.261))
])

In [5]:
train_data = CIFAR10(root='./',train=True,download=True,transform=transforms)
test_data = CIFAR10(root='./',train=False,download=True,transform=transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = CNN(10)

model.to(device)

Files already downloaded and verified
Files already downloaded and verified


CNN(
  (block1): BasicBlock(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): BasicBlock(
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4096, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=256,

In [6]:
# SummaryWriter 인스턴스 생성
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
lr = 1e-3

optim = Adam(model.parameters(),lr=lr)

for epoch in range(100):
    for data,label in train_loader:
        optim.zero_grad()

        preds = model(data.to(device))

        loss = nn.CrossEntropyLoss()(preds,label.to(device))
        loss.backward()
        optim.step()

    print(f'epoch : {epoch+1} loss:{loss.item()}')
    writer.add_scalar("Loss/train", loss.item(), epoch+1 )

epoch : 1 loss:0.8389108180999756
epoch : 2 loss:1.526585340499878
epoch : 3 loss:0.8053567409515381
epoch : 4 loss:1.1447548866271973
epoch : 5 loss:0.9441565871238708
epoch : 6 loss:0.5827739238739014
epoch : 7 loss:0.8425121307373047
epoch : 8 loss:0.970990002155304
epoch : 9 loss:0.5536876320838928
epoch : 10 loss:0.4515862464904785
epoch : 11 loss:0.438700795173645
epoch : 12 loss:0.3833353519439697
epoch : 13 loss:0.4236484169960022
epoch : 14 loss:0.5880935788154602
epoch : 15 loss:0.7043348550796509
epoch : 16 loss:0.2540919780731201
epoch : 17 loss:0.9258472919464111
epoch : 18 loss:0.3941402733325958
epoch : 19 loss:1.0268940925598145
epoch : 20 loss:0.20321154594421387
epoch : 21 loss:0.7881424427032471
epoch : 22 loss:0.31635287404060364
epoch : 23 loss:0.27432072162628174
epoch : 24 loss:0.5567041039466858
epoch : 25 loss:0.23771774768829346
epoch : 26 loss:0.10682253539562225
epoch : 27 loss:0.8278865814208984
epoch : 28 loss:0.3112070560455322
epoch : 29 loss:0.731756865

In [8]:
torch.save(model.state_dict(), 'cifar-cnn.pth')

In [9]:
num_corr =  0

model.eval()
with torch.no_grad():
    for data,label in test_loader:
        pred = model(data.to(device))
        preds = pred.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr
num_corr/len(test_data)

0.8219

In [11]:
tensorboard --logdir=/tmp/sample

In [13]:
# 쓰기 종료
writer.close()

#코랩 안에서 띄우기위함
%load_ext tensorboard
#경로로 불러오기
%tensorboard --logdir=runs --port 6060
# %tensorboard --logdir [경로] --port 6060

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
